In [51]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor

In [31]:
housedf = pd.read_csv('BostonHousing.csv',index_col='Id')

In [32]:
housedf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 80 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-

In [33]:
housedf.sample(10)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1310,20,RL,NaN,7153,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,1Fam,1Story,6,5,1991,1991,Gable,CompShg,HdBoard,HdBoard,BrkFace,88.0,TA,TA,CBlock,Gd,TA,No,GLQ,1200,Unf,0,78,1278,GasA,Gd,Y,SBrkr,1294,0,0,1294,1,0,2,0,3,1,Gd,6,Typ,0,NaN,Attchd,1991.0,RFn,2,496,TA,TA,Y,112,51,0,0,0,0,NaN,GdWo,NaN,0,6,2008,WD,Normal,179200
137,20,RL,NaN,10355,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1967,1967,Gable,CompShg,MetalSd,MetalSd,BrkFace,196.0,TA,TA,CBlock,TA,TA,No,BLQ,695,Unf,0,519,1214,GasA,TA,Y,SBrkr,1214,0,0,1214,0,0,2,0,3,1,TA,5,Typ,1,Fa,Attchd,1967.0,RFn,1,318,TA,TA,Y,0,111,0,0,0,0,NaN,NaN,NaN,0,7,2007,WD,Normal,143000
1420,20,RL,NaN,16381,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,6,5,1969,1969,Gable,CompShg,Plywood,Plywood,BrkFace,312.0,Gd,Gd,CBlock,TA,TA,Av,Rec,1110,Unf,0,734,1844,GasA,Gd,Y,SBrkr,1844,0,0,1844,1,0,2,0,3,1,Gd,7,Typ,1,TA,Attchd,1969.0,RFn,2,540,TA,TA,Y,0,73,216,0,0,0,NaN,NaN,NaN,0,12,2006,WD,Normal,223000
949,60,RL,65.0,14006,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,144.0,Gd,TA,PConc,Gd,TA,NaN,Unf,0,Unf,0,936,936,GasA,Ex,Y,SBrkr,936,840,0,1776,0,0,2,1,3,1,Gd,7,Typ,1,TA,Attchd,2002.0,RFn,2,474,TA,TA,Y,144,96,0,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Normal,192500
583,90,RL,81.0,11841,Grvl,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,Duplex,SFoyer,6,5,1990,1990,Gable,CompShg,HdBoard,HdBoard,BrkFace,104.0,TA,Gd,CBlock,Gd,TA,Av,GLQ,816,Unf,0,0,816,GasA,TA,Y,SBrkr,816,0,0,816,1,0,1,0,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Y,0,32,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,118500
671,60,RL,64.0,8633,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,CollgCr,Norm,Norm,1Fam,2Story,6,5,2005,2005,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,193,Unf,0,545,738,GasA,Ex,Y,SBrkr,738,738,0,1476,1,0,2,1,3,1,Gd,7,Typ,0,NaN,Attchd,2005.0,Fin,2,540,TA,TA,Y,100,35,0,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,173500
370,20,RL,NaN,9830,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1959,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,Gd,CBlock,TA,TA,No,ALQ,72,Rec,258,733,1063,GasA,Ex,Y,SBrkr,1287,0,0,1287,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1997.0,Fin,2,576,TA,TA,Y,364,17,0,0,182,0,NaN,NaN,NaN,0,3,2010,WD,Normal,162000
869,60,RL,NaN,14762,Pave,NaN,IR2,Lvl,AllPub,Corner,Gtl,Gilbert,Feedr,Norm,1Fam,2Story,5,6,1948,1950,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,Gd,Y,SBrkr,1547,720,53,2320,0,0,2,0,2,1,TA,7,Typ,1,TA,Attchd,1979.0,Unf,2,672,TA,TA,P,120,144,0,0,0,0,NaN,NaN,NaN,0,5,2006,WD,Normal,169000
536,190,RL,70.0,7000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,2fmCon,2Story,5,7,1910,1991,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,GLQ,969,Unf,0,148,1117,GasA,TA,Y,SBrkr,820,527,0,1347,1,0,1,0,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,85,0,148,0,0,0,NaN,NaN,NaN,0,1,2008,WD,Normal,107500


In [34]:
X = housedf.drop('SalePrice', axis =1)
y = housedf.SalePrice

In [35]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=9)

In [36]:
X_train.shape

(1168, 79)

In [37]:
y_train.shape

(1168,)

In [38]:
X_val.shape

(292, 79)

In [39]:
y_val.shape

(292,)

In [58]:
#create list of categorical columns and numeric columns
categorical_cols =[]
numerical_cols=[]

for col in X_train.columns:
    if  X_train[col].nunique() <= 10 and X_train[col].dtype == "object":
        categorical_cols.append(col)
    elif X_train[col].dtype in ['int64','float64']:
        numerical_cols.append(col)

In [59]:
len(categorical_cols)

40

In [60]:
len(numerical_cols)

36

In [61]:
selected_cols = categorical_cols + numerical_cols
X_train = X_train[selected_cols]
X_val = X_val[selected_cols]

In [62]:
X_train.shape

(1168, 76)

In [63]:
X_val.shape

(292, 76)

In [64]:
#preprocessing numerical data
numerical_transformer = SimpleImputer(strategy='mean')

#preprocessing categorical data
categporical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categporical_transformer, categorical_cols)
])

model = XGBRegressor(n_estimators=1000, learning_rate=0.01, random_state=11, verbosity=1)

pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

pipe.fit(X_train, y_train)

pred = pipe.predict(X_val)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[21:55:21] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [65]:
mean_absolute_error(pred, y_val)

14926.561911386987

In [66]:
dfactualvspredicted = pd.DataFrame({'Actual':y_val, 'Predicted':pred})

In [68]:
dfactualvspredicted.sample(10)

,Actual,Predicted
Id,,
260,97000,107046.976562
1075,194000,189735.171875
419,126000,108385.164062
869,169000,159333.750000
6,143000,152948.343750
1222,134000,122856.648438
588,137000,130476.578125
1453,145000,133280.843750
28,306000,284253.531250
